Prepare the dataset and train a model on the data.

In [1]:
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(project_root)
from examples.mia.text_mia.utils.tabds_data_preparation import *
from examples.mia.text_mia.utils.tabds_model_preparation import *

/home/amandus/miniconda3/envs/leakpro/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_to_datafolder = os.path.join(os.getcwd(), "tab_data/")
dataset = preprocess_tab_dataset(path_to_datafolder, create_new=False)

train_loader, test_loader = get_tab_dataloaders(dataset, train_fraction=0.1, 
test_fraction=0.1)

### Generate target model 

if not os.path.exists("target"):
    os.makedirs("target")


n_classes = 3 # Case dependent, is equal to 2*number of masktypes + 1
pretrained_model_name = "allenai/longformer-base-4096"
model = TABBERT(pt_model= pretrained_model_name, num_classes=n_classes)
n_epochs = 1


train_acc, train_loss, test_acc, test_loss = create_trained_model_and_metadata(model = model, 
                                                                               train_loader = train_loader, 
                                                                               test_loader = test_loader, 
                                                                               epochs = n_epochs)

Loading local dataset
Training
Epoch 1 started


100%|██████████| 40/40 [00:18<00:00,  2.12it/s]


Evaluating


100%|██████████| 40/40 [00:04<00:00,  9.63it/s]


Create LeakPro objects and run attack

In [3]:
from tabds_handler import TABInputHandler

from leakpro import LeakPro

# Read the config file
config_path = "audit.yaml"

# Prepare leakpro object
leakpro = LeakPro(TABInputHandler, config_path)

# Run the audit 
leakpro.run_audit()

2024-10-31 09:19:25,329 INFO     Target model blueprint created from TABBERT in utils/tabds_model_preparation.py.
2024-10-31 09:19:25,331 INFO     Loaded target model metadata from ./target/model_metadata.pkl
/home/amandus/msc-code/LeakPro/leakpro/input_handler/handler_setup.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` f

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [2], [2, 4096]

In [8]:
leakpro.run_audit()

2024-10-30 13:06:53,308 INFO     Preparing attack: lira
2024-10-30 13:47:29,015 INFO     Number of existing models exceeds or equals the number of models to create
2024-10-30 13:48:00,664 INFO     Loading shadow model 0
/home/amandus/msc-code/LeakPro/leakpro/attacks/utils/model_handler.py:162: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/amandus/miniconda3/envs/leakpro/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_720175/2014151989.py", line 1, in <module>
    leakpro.run_audit()
  File "/home/amandus/msc-code/LeakPro/leakpro/leakpro.py", line 131, in run_audit
    audit_results = self.attack_scheduler.run_attacks()
  File "/home/amandus/msc-code/LeakPro/leakpro/attacks/attack_scheduler.py", line 63, in run_attacks
    result = attack.run_attack()
  File "/home/amandus/msc-code/LeakPro/leakpro/attacks/mia_attacks/lira.py", line 266, in run_attack
  File "/home/amandus/msc-code/LeakPro/leakpro/attacks/mia_attacks/lira.py", line 216, in get_std
    # Fixed/Global variance calculation.
  File "/home/amandus/msc-code/LeakPro/leakpro/attacks/mia_attacks/lira.py", line 232, in _fixed_variance
    if is_in and not self.online:
IndexError: boolean index did not ma